# Epey.com Web Scraping Project For Scrape Smartphone Data

This project uses Selenium to fetch HTML codes via the Microsoft Edge browser, BeautifulSoup to extract data from the HTML codes, pandas to store the extracted data in a DataFrame, and convert it into an Excel file.

If you do not have these packages installed, you should install these packages first.

pip install selenium

pip install beautifulsoup4

pip install pd


We imported the necessary packages

In [5]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from bs4 import BeautifulSoup
import pandas as pd

We enter how many pages we want to scrape into the page variable.

In [6]:
page = 70

url = []
for i in range(1, page+1):
    url.append("https://www.epey.com/akilli-telefonlar/"+str(i)+"/")

Our function to make the price value float

In [7]:
def fix_price(price):
    price_fixed = price.split(' ')[0]
    price_fixed = price_fixed.replace('.', '').replace(',', '.')
    return float(price_fixed)

We are now ready for the scrape process

In [8]:
options = EdgeOptions()
options.use_chromium = True
options.headless = True

driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)

driver.get(url[0])

driver.implicitly_wait(10)

html = driver.page_source

soup = BeautifulSoup(html, "html.parser")

driver.quit()

table = soup.find("div", class_="listele table")
table_header = table.find("ul", class_="baslik row")
columns = table_header.find_all("li")
table_columns = [col.text for col in columns]

Tablo = pd.DataFrame(columns=table_columns)

row = 1

for i in range(1, page+1):

    for phone in table.find_all("ul", class_="metin row")[1:]:
        Row = []

        Row.append(phone.find("a", class_="urunadi").text)

        try:
            fiyat = phone.find("li", class_="fiyat cell").find("a").text
            Row.append(fix_price(fiyat))
        except:
            Row.append(0)

        Row.append(phone.find("li", class_="ozellik ozellik1 cell").text)
        Row.append(phone.find("li", class_="ozellik ozellik21 cell").text)
        Row.append(phone.find("li", class_="ozellik ozellik14 cell").text)
        Row.append(phone.find("li", class_="ozellik ozellik7 cell").text)
        try:
            Row.append(int(phone.find("li", class_="puan cell").find("div", class_="circliful").get("data-percent")))
        except:
            Row.append(0)
        Tablo.loc[row - 1] = Row
        row = row + 1
    if i == page:
        break
    options = EdgeOptions()
    options.use_chromium = True
    options.headless = True

    driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)

    driver.get(url[i])

    driver.implicitly_wait(10)

    html = driver.page_source

    soup = BeautifulSoup(html, "html.parser")

    driver.quit()

    table = soup.find("div", class_="listele table")

Tablo.to_excel("epey.xlsx", index=False)
